------
# Singapore Neighborhood Segmentation
- Link to repository can be found here: https://github.com/ianap/sg-neighborhood/tree/master

In [1]:
import pandas as pd
from importlib import reload
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
import pandas.io.json._normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
import numpy as np # library to handle data in a vectorized manner

print('Libraries imported.')

Libraries imported.


In [2]:
import sys  

# reload(sys)  
# sys.setdefaultencoding('utf8')\
if sys.version[0] == '2':
    reload(sys)
    sys.setdefaultencoding("utf-8")

In [3]:
neighborhoods = pd.read_csv('singapore_geo.csv')

In [4]:
print('The dataframe has {} postcodes and {} neighborhoods.'.format(
        len(neighborhoods['Postcode'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 85265 postcodes and 155744 neighborhoods.


In [5]:
neighborhoods.head(15)

,Unnamed: 0,Postcode,Neighborhood,Latitude,Longitude
0,0,18906,SINGAPORE CHINESE CULTURAL CENTRE,1.275829,103.849576
1,1,18907,TEMPORARY SITE OFFICE,1.274950,103.851665
2,3,18915,TEMPORARY SITE OFFICE,1.273682,103.860075
3,4,18925,CITIBANK TRADE_BRANCH,1.276424,103.854759
4,5,18925,DBS Marina Bay MRT Station,1.276427,103.854598
5,6,18925,MARINA BAY MRT STATION,1.276311,103.854605
6,7,18925,MARINA BAY MRT STATION (NS27),1.276427,103.854598
7,8,18925,OCBC Marina Bay MRT,1.276427,103.854598
8,9,18925,UOB Marina Bay MRT Station,1.276427,103.854598
9,10,18926,MARINA BAY MRT STATION (CE2),1.276251,103.855447


### random selection of 500 neighborhoods

In [6]:
neighborhoods_subset = neighborhoods.sample(500)
neighborhoods_subset.head()

,Unnamed: 0,Postcode,Neighborhood,Latitude,Longitude
32452,48779,408659,KAMPONG UBI INDUSTRIAL ESTATE,1.330008,103.897403
31268,46239,389294,GEYLANG CONSERVATION AREA,1.312214,103.876505
115362,180917,554330,AFFINITY AT SERANGOON,1.365651,103.874302
146325,219314,768769,CHEE LENG LIAN HE MIAO,1.443503,103.835838
12422,16114,208987,PETAIN ROAD/TYRWHITT ROAD CONSERVATION AREA,1.312086,103.859439


In [7]:
# use .loc method to remove unnamed columns in df
neighborhoods_subset= neighborhoods_subset.loc[:, ~neighborhoods_subset.columns.str.contains("^Unnamed")]
print(neighborhoods_subset.shape)
neighborhoods_subset.head()

(500, 4)


,Postcode,Neighborhood,Latitude,Longitude
32452,408659,KAMPONG UBI INDUSTRIAL ESTATE,1.330008,103.897403
31268,389294,GEYLANG CONSERVATION AREA,1.312214,103.876505
115362,554330,AFFINITY AT SERANGOON,1.365651,103.874302
146325,768769,CHEE LENG LIAN HE MIAO,1.443503,103.835838
12422,208987,PETAIN ROAD/TYRWHITT ROAD CONSERVATION AREA,1.312086,103.859439


In [8]:
neighborhoods_subset.shape

(500, 4)

#### Some neighborhoods correspond to the Conservation Area, which is multiple locations
- we will exclude all Conservation Areas from our analysis.

In [9]:
to_drop= ["CONSERVATION", "AREA"]
subset = neighborhoods_subset[~neighborhoods_subset.Neighborhood.str.contains("CONSERVATION AREA")]
subset.shape

(460, 4)

We have removed 25 neighborhoods.

In [10]:
subset.head()

,Postcode,Neighborhood,Latitude,Longitude
32452,408659,KAMPONG UBI INDUSTRIAL ESTATE,1.330008,103.897403
115362,554330,AFFINITY AT SERANGOON,1.365651,103.874302
146325,768769,CHEE LENG LIAN HE MIAO,1.443503,103.835838
121663,570233,BISHAN NORTH COMMUNITY CENTRE,1.358588,103.845629
140887,728857,SUNGEI KADUT INDUSTRIAL ESTATE,1.428835,103.749767


### clean data

In [11]:
occurrences = subset.groupby('Neighborhood').count().reset_index()
occurrences.head()

,Neighborhood,Postcode,Latitude,Longitude
0,ABC HOSTEL,1,1,1
1,AFFINITY AT SERANGOON,1,1,1
2,AIRVIEW PARK,1,1,1
3,ALTEZ,1,1,1
4,ANCHORVALE COVE,1,1,1


In [12]:
occurrences.sort_values(by="Latitude",ascending=False).head()

,Neighborhood,Postcode,Latitude,Longitude
251,SERANGOON GARDEN ESTATE,12,12,12
203,OPERA ESTATE,10,10,10
332,WEST COAST GARDENS,7,7,7
88,FRANKEL ESTATE,6,6,6
249,SENNETT ESTATE,5,5,5


In [13]:
tmp = occurrences[occurrences['Latitude']>1]
print(tmp.shape)
display(tmp.head())

(58, 4)


,Neighborhood,Postcode,Latitude,Longitude
6,APOLLO GARDENS,2,2,2
10,BAN GUAN PARK,4,4,4
11,BARTLEY RISE ESTATE,3,3,3
13,BEDOK PESARI,2,2,2
14,BEDOK RIA,3,3,3


In [14]:
tmp2 = occurrences[occurrences['Latitude']==1]
tmp2.sample(5)

,Neighborhood,Postcode,Latitude,Longitude
264,SPC HAVELOCK,1,1,1
119,HDB-SERANGOON NORTH ESTATE,1,1,1
78,EVERGREEN GARDEN,1,1,1
291,THE CENTRIS,1,1,1
223,PRESIDENT GARDENS,1,1,1


#### 58 neighborhoods occurred more than once. We will take in only one iteration

In [15]:
def subset_df_2(input):
    n=[]
    lat=[]
    long=[]
    for name in input.Neighborhood:
        n.append(name)
        coords = subset[subset['Neighborhood']==name][["Latitude", "Longitude"]]
        lat.append(coords.iloc[0,0])
        long.append(coords.iloc[0,1])
    
    df_coord= pd.DataFrame({"Neighborhood":n, "Latitude":lat, "Longitude":long})
    df_coord= df_coord[["Neighborhood", "Latitude", "Longitude"]]
    
    return df_coord

In [19]:
new_df = subset_df_2(tmp)
new_df.head()

,Neighborhood,Latitude,Longitude
0,APOLLO GARDENS,1.338189,103.958424
1,BAN GUAN PARK,1.321497,103.787284
2,BARTLEY RISE ESTATE,1.345329,103.873163
3,BEDOK PESARI,1.324030,103.955718
4,BEDOK RIA,1.325454,103.954230


In [21]:
new_df2 = subset_df_2(tmp2)
new_df2.head()

,Neighborhood,Latitude,Longitude
0,ABC HOSTEL,1.303762,103.859237
1,AFFINITY AT SERANGOON,1.365651,103.874302
2,AIRVIEW PARK,1.342745,103.774609
3,ALTEZ,1.274618,103.844320
4,ANCHORVALE COVE,1.398462,103.889969


In [24]:
data = pd.concat([new_df, new_df2], ignore_index=True)
data.shape
# -append method is deprecated. -pd.concat([df1, df2]) is the new approach 

(345, 3)

# 3. Create a map of Singapore with neighborhoods superimposed on top.¶
#### Use geopy library to get the latitude and longitude values of Singapore.

In [25]:
n_subset = pd.read_csv("sg_random_samples_clean.csv")

In [26]:
# obtain coordinates of singapore
#Nominatim(user_agent="my-application")
#nom = Nominatim()
address = 'Singapore'
geolocator = Nominatim(user_agent="aj656@cornell.edu")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Singapore\'s geograpical coordinates {}, {}.'.format(latitude, longitude))

Singapore's geograpical coordinates 1.357107, 103.8194992.


In [27]:
map_sg = folium.Map(location=[latitude, longitude], zoom_start=10) 
for lat, lng, neighborhood in zip(n_subset['Latitude'], n_subset['Longitude'], n_subset['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.CircleMarker(
        [lat,lng], 
        radius=5, 
        popup=label, 
        color='blue', 
        fill=True, 
        fill_color="#3186c",
        fill_opacity=0.7,
        parse_html=False
    ).add_to(map_sg)
map_sg

------
# Using FourSquare API

In [77]:
CLIENT_ID = 'FUSI3FALTRTMCFCO0I0T322WWI21J5ZJMAM1SL2G2XVXTCRT' # your Foursquare ID
CLIENT_SECRET = '5M5YCWZ2SE4IU1M3CFHE5S2GCVKGRIHM4O51ATLL3H4RXD5C' # your Foursquare Secret
ACCESS_TOKEN = 'O2WL0RVC0VHTEFIT0SFR1ZVUTPT324UR1TJCJSKDOCKIFPAF' # access token
VERSION = '20240419' # Foursquare API version

In [78]:
n_subset=n_subset.loc[:, ~n_subset.columns.str.contains('^Unnamed')]
print(n_subset.shape)
n_subset.head()

(353, 3)


,Neighborhood,Latitude,Longitude
0,BEDOKVILLE,1.323681,103.947566
1,BRADDELL HEIGHTS ESTATE,1.349106,103.865046
2,BUKIT LOYANG ESTATE,1.361056,103.964245
3,CAPITOL PARK,1.328713,103.817978
4,CASHEW GREEN,1.373039,103.770301


In [79]:
n_subset.loc[0, 'Neighborhood']

'BEDOKVILLE'

#### Obtain neighborhood latitude and longitude values

In [80]:
n_latitude = n_subset.loc[0, 'Latitude']
n_longitude= n_subset.loc[0, 'Longitude']
n_name= n_subset.loc[0, 'Neighborhood']
print('Latitude and longitude values of {} are {}, {}.'.format(n_name, 
                                                               n_latitude, 
                                                               n_longitude))

Latitude and longitude values of BEDOKVILLE are 1.32368059857, 103.947565974.


#### Obtain top 100 venues in Bedokville (radius of 500 meters)

In [81]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500
# create URL
url = 'https://api.foursquare.com/v2/venues/search?&client_id=CLIENT_ID&client_secret=CLIENT_SECRET&oauth_token=ACCESS_TOKEN&v=VERSION&ll=n_latitude,n_longitude&radius=RADIUS&limit=LIMIT'.format(
   #  'https://api.foursquare.com/v2/venues/search?&client_id=CLIENT_ID&client_secret=CLIENT_SECRET&v=YYYYMMDD'
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    n_latitude, 
    n_longitude, 
    radius, 
    LIMIT)
url
# https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}
# https://api.foursquare.com/v2/venues/search?ll=40.7,-74&client_id=CLIENT_ID&client_secret=CLIENT_SECRET&v=YYYYMMDD

'https://api.foursquare.com/v2/venues/search?&client_id=CLIENT_ID&client_secret=CLIENT_SECRET&oauth_token=ACCESS_TOKEN&v=VERSION&ll=n_latitude,n_longitude&radius=RADIUS&limit=LIMIT'

In [83]:
response = requests.get(url) # response object
results = response.json() # results 
results

{'meta': {'code': 401,
  'errorType': 'invalid_auth',
  'errorDetail': 'OAuth token invalid or revoked.',
  'requestId': '66232652855a594f0f65ab69'},
 'response': {}}

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

## How many venues were returned?

In [ ]:
(nearby_venues.shape[0])

# Explore neighborhoods in Singapore

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
res=getNearbyVenues(names=neighborhoods_subset.Neighborhood, latitudes=neighborhoods_subset.Latitude,
                    longitudes=neighborhoods_subset.Longitude)
res

In [ ]:
res.to_csv("neighborhood_samples_venues_500m.csv")
singapore_venues = res
(singapore_venues.shape)

## How many venues were returned for each neighborhood?

In [ ]:
sg_grouped=singapore_venues.groupby('Neighborhood').count()
sg_grouped.head()

In [ ]:
sg_grouped.sort_values('Venue', ascending=False)

In [ ]:
# one hot encoding
sg_onehot = pd.get_dummies(singapore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sg_onehot['Neighborhood'] = singapore_venues['Neighborhood'] 

# move neighborhood column to the first column
cols = sg_onehot.columns.tolist()
cols.insert(0, cols.pop(cols.index('Neighborhood')))

sg_onehot = sg_onehot.reindex(columns= cols)
sg_onehot.head()

## Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
sg_grouped = sg_onehot.groupby('Neighborhood').mean().reset_index()
sg_grouped.head()

In [ ]:
sg_grouped.describe()

In [ ]:
(sg_grouped.sum(axis=0).sort_values())

### The most common venue's categories are Bus Station, Coffee Shop, Food Court, Cafe, Chinese Restaurant.
---
## CASE 1: Explore all neighborhoods which is near to the art gallery.

In [ ]:
#get a list of a neighboorhoods names
art_buildings=sg_onehot[sg_onehot["Art Gallery"]>0].Neighborhood.unique().tolist()
art_buildings

In [ ]:
# subset all geo locations and highlight on the map:
sg_art_neighboors = singapore_venues[(singapore_venues["Neighborhood"].isin(art_buildings))]
sg_art_neighboors=sg_art_neighboors.iloc[:,0:3]
sg_art_neighboors.head()

In [ ]:
# dropping duplicates:
sg_art_neighboors=sg_art_neighboors.drop_duplicates()
sg_art_neighboors.shape

In [ ]:
map_sg = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(sg_art_neighboors['Neighborhood Latitude'], sg_art_neighboors['Neighborhood Longitude'], sg_art_neighboors['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sg)  
    
map_sg

As expected, most places, which are close to some art gallery, are located in the Central parts of Singapore.

### Cluster Neighborhoods

In [ ]:
# set number of clusters
kclusters = 8

sg_grouped_clustering = sg_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sg_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

## Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = sg_grouped['Neighborhood']

for ind in np.arange(sg_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sg_grouped.iloc[ind, :], num_top_venues)


print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.head()

In [ ]:
o_drop = ['CONSERVATION', 'AREA']
neighborhoods_venues_sorted=neighborhoods_venues_sorted[~neighborhoods_venues_sorted.Neighborhood.str.contains("CONSERVATION AREA")]
neighborhoods_venues_sorted.shape

##### Not all neighborhoods have a venues. So we will removed those from our neighborhoods_subset

In [ ]:
names_k=neighborhoods_venues_sorted.Neighborhood.unique().tolist()
len(names_k)

In [ ]:
neighborhoods_subset.shape

#### Fix duplicate data

In [ ]:
sg_merged = neighborhoods_subset

# add clustering labels
sg_merged['Cluster Labels'] = kmeans.labels_

# merge sg_grouped with toronto_data to add latitude/longitude for each neighborhood
sg_merged = sg_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

sg_merged.head() # check the last columns!

### Visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
#rainbow = ["#FF33F6","#33E0FF","#7A33FF","#FF7D33","#7E0548","#2DBB1F","#05127E"]
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sg_merged['Latitude'], sg_merged['Longitude'], sg_merged['Neighborhood'], sg_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=4,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.5).add_to(map_clusters)
       
map_clusters